In [ ]:
# Initialize color map (31 colors for 0-30 meters)
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

cmap = plt.get_cmap('jet')
cmap_colors = (cmap(np.linspace(0, 1, 31)) * 255).astype(np.uint8)

def project_points(T_cam_lidar, xyz_points, img, camera, img_path, num_beam):
  """Project 3D points to image plane with enhanced logic"""
  # Get transformation matrix
  R, t = T_cam_lidar[:3, :3], T_cam_lidar[:3, 3]
  
  # Transform points (vectorized)
  xyz_cam = (xyz_points @ R.T) + t
  
  # Initialize depth image with infinity
  depth_img = np.full((camera.height, camera.width), np.inf, dtype=np.float32)
  
  # Vectorized projection
  valid_mask = np.ones(len(xyz_cam), dtype=bool)
  uvs = np.zeros((len(xyz_cam), 2))
  
  for i, p in enumerate(xyz_cam):
    success, uv = camera.project(p)
    if success:
      uvs[i] = uv
    valid_mask[i] = success
  
  # Filter valid points
  uvs = uvs[valid_mask].astype(int)
  depths = xyz_cam[valid_mask, 2]
  colors = cmap_colors[np.clip(depths.astype(int), 0, 30), :]
  
  # Update depth image (keep closest points)
  for (u, v), d in zip(uvs, depths):
    if 0 <= u < img.shape[1] and 0 <= v < img.shape[0]:
      if d < depth_img[v, u]:
        i = int(min(d, 20.0))
        color = (int(cmap_colors[i, 0]), int(cmap_colors[i, 1]), int(cmap_colors[i, 2]))        
        depth_img[v, u] = d
        cv2.circle(img, (u, v), 1, color=color, thickness=-1)

  # Save color projection
  image_path = Path(img_path.replace('image', f'color_proj_image_{num_beam}'))
  image_path.parent.mkdir(parents=True, exist_ok=True)
  Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).save(str(image_path))

  # Save depth image (convert to uint16 in millimeters)
  depth_img = np.where(np.isfinite(depth_img), depth_img * 1000, 0).astype(np.uint32)
  image_path = Path(img_path.replace('image', f'depth_image_{num_beam}'))
  image_path.parent.mkdir(parents=True, exist_ok=True)
  Image.fromarray(depth_img).save(str(image_path))

In [ ]:
from data_loader.calib.intrinsic_extrinsic_loader import IntrinsicExtrinsicLoader

def generate_depthmap_to_kitti360(dataset_path, calib_path, platform, sequence_name, algorithm):
  ##### Set up the output data path
  kitti360_path = os.path.join(dataset_path, sequence_name, 'kitti360')

  ##### Set up the message topic list for different platforms
  # Platform
  if platform == 'handheld':
    from cfg.dataset.cfg_handheld import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_handheld import dataset_rostopic_msg_frameid_dict
  elif platform == 'ugv':
    from cfg.dataset.cfg_ugv import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_ugv import dataset_rostopic_msg_frameid_dict
  elif platform == 'legged':
    from cfg.dataset.cfg_legged import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_legged import dataset_rostopic_msg_frameid_dict
    if sequence_name == 'legged_grass00':
      dataset_sensor_frameid_dict, dataset_rostopic_msg_frameid_dict = \
        filter_sensor('event', dataset_sensor_frameid_dict, dataset_rostopic_msg_frameid_dict)
  elif platform =='vehicle':
    from cfg.dataset.cfg_vehicle import dataset_sensor_frameid_dict
    from cfg.dataset.cfg_vehicle import dataset_rostopic_msg_frameid_dict

  ##### Set up the sensor configuration
  int_ext_loader = IntrinsicExtrinsicLoader(is_print=False)
  int_ext_loader.load_calibration(calib_path=calib_path, sensor_frameid_dict=dataset_sensor_frameid_dict)
  print('Finish loading parameters')

  # Description: Generate depth map with enhanced projection logic
  num_beam_input = 128
  # Main processing loop
  for frame_id in range(100000):
    # Load point cloud data
    pcd_path = os.path.join(kitti360_path, 'ouster00_undistort/points/data', f'{frame_id:06d}.pcd')
    if not os.path.exists(pcd_path): break
    
    pcd = o3d.io.read_point_cloud(pcd_path)
    if not pcd.has_points(): continue
    xyz_points = np.asarray(pcd.points)

    for num_beam_output in [128, 64, 32, 16, 8, 4]:
      r = np.sqrt(xyz_points[:, 0]**2 + xyz_points[:, 1]**2) + 1e-8
      theta = np.arctan(xyz_points[:, 2] / r)
      sorted_indices = np.argsort(theta)
      sorted_points = xyz_points[sorted_indices]
      groups = np.array_split(sorted_points, num_beam_input)
      xyz_points = np.concatenate(groups[::int(num_beam_input / num_beam_output)], axis=0)

      # Camera processing configuration
      camera_configs = [
          # Frame cameras
          {'type': 'frame', 'suffix': '00', 'cam_spec': ('left', 'right')},
          {'type': 'frame', 'suffix': '01', 'cam_spec': ('right', 'right')},
          # Event cameras
          {'type': 'event', 'suffix': '00', 'cam_spec': ('left',)},
          {'type': 'event', 'suffix': '01', 'cam_spec': ('right',)}
      ]

      for config in camera_configs:
          # Generate camera parameters
          if config['type'] == 'frame':
              cam_side = 'vehicle_' if platform == 'vehicle' else ''
              cam_id = f"{cam_side}frame_{config['cam_spec'][0]}_camera"
              img_folder = f"{cam_side}frame_cam{config['suffix']}"
          else:
              cam_id = f"event_{config['cam_spec'][0]}_camera"
              img_folder = f"event_cam{config['suffix']}"

          # Build image path
          img_path = os.path.join(
              kitti360_path,
              f'{img_folder}/image/data',
              f'{frame_id:06d}.png'
          )

          # Load and validate image
          img = cv2.imread(img_path)
          if img is None:
              print(f"Missing image: {img_path}")
              continue

          # Get camera parameters and transformation
          camera = int_ext_loader.sensor_collection[cam_id]
          T_cam_lidar = int_ext_loader.tf_graph.get_relative_transform(camera.frame_id, 'body_imu')
          project_points(T_cam_lidar, xyz_points, img.copy(), camera, img_path, num_beam_output)
          

In [ ]:
import os
import cv2
import numpy as np
import open3d as o3d
from PIL import Image
from cfg.dataset.cfg_sequence import dataset_sequence_calib_used_dict

dataset_path = '/Rocket_ssd/dataset/data_FusionPortable/sensor_data'
algorithms = ['fastlio2']

for sequence_name, values in dataset_sequence_calib_used_dict.items():
  for algorithm in algorithms:
    platform, calib_folder, used = values[0], values[1], values[2]
    if used:
      calib_path = os.path.join(dataset_path, '../calibration_files', calib_folder, 'calib')
      print('Start processing platform: {} sequence: {} with algorithm_result: {}'.format(platform, sequence_name, algorithm))
      
      generate_depthmap_to_kitti360(dataset_path, calib_path, platform, sequence_name, algorithm)
      print('Finish generating depth images')